<a href="https://colab.research.google.com/github/SBhat2615/pcap-mini-project/blob/main/colab-workspace.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# To show that if there is cuda tookit installed
!ls /usr/local

bin    cuda	cuda-11.8  games	       include	lib64	   man	 share
colab  cuda-11	etc	   _gcs_config_ops.so  lib	licensing  sbin  src


In [3]:
# To show that if we have the nvcc command
!which nvcc

/usr/local/cuda/bin/nvcc


In [4]:
# To show the property of the nvidia card(On my one, I use the K80)
!nvidia-smi

Thu Apr 27 13:24:53 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

#Download Image

In [5]:
!wget https://github.com/SBhat2615/pcap-mini-project/blob/main/image.jpg

--2023-04-27 13:25:04--  https://github.com/SBhat2615/pcap-mini-project/blob/main/image.jpg
Resolving github.com (github.com)... 140.82.121.3
Connecting to github.com (github.com)|140.82.121.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘image.jpg’

image.jpg               [ <=>                ] 135.71K  --.-KB/s    in 0.02s   

2023-04-27 13:25:04 (5.75 MB/s) - ‘image.jpg’ saved [138969]



In [6]:
!ls -l

total 140
-rw-r--r-- 1 root root 138969 Apr 27 13:25 image.jpg
drwxr-xr-x 1 root root   4096 Apr 25 13:34 sample_data


In [7]:
!pwd

/content


# Image to GrayScale

In [8]:
"""
from PIL import Image

# Open the image file
img = Image.open('/content/image.jpg')

# Convert to grayscale
gray_img = img.convert('L')

# Display the grayscale image
gray_img.show()

# Save the grayscale image
gray_img.save('/content/imageGray.jpg')
"""

"\nfrom PIL import Image\n\n# Open the image file\nimg = Image.open('/content/image.jpg')\n\n# Convert to grayscale\ngray_img = img.convert('L')\n\n# Display the grayscale image\ngray_img.show()\n\n# Save the grayscale image\ngray_img.save('/content/imageGray.jpg')\n"

# Load Image

In [9]:
%%writefile loadImg.c
#include <stdio.h>
#include <stdlib.h>

// Define Image Height & Width
#define IMG_WIDTH 512
#define IMG_HEIGHT 512

int main() {
    FILE *fp;
    char *filename = "./image.jpg";
    unsigned char img[IMG_HEIGHT][IMG_WIDTH];
    int i, j;

    // Open image file
    if ((fp = fopen(filename, "rb")) == NULL) {
        printf("Error: Image file not found\n");
        return 1;
    }

    // Skip image header
    fseek(fp, 8, SEEK_SET);

    // Read image pixel values
    for (i = 0; i < IMG_HEIGHT; i++) {
        for (j = 0; j < IMG_WIDTH; j++) {
            img[i][j] = fgetc(fp);
        }
    }

    // Close image file
    fclose(fp);

    // Print image data
    /*
    for (i = 0; i < IMG_HEIGHT; i++) {
        for (j = 0; j < IMG_WIDTH; j++) {
            printf("%u ", img[i][j]);
        }
        printf("\n");
    }
    */
    printf("Image Width : %d\n", IMG_WIDTH);
    printf("Image Height : %d\n", IMG_HEIGHT);

    printf("Image Matrix:\n");
    for (i = 0; i < 25; i++) {
        for (j = 0; j < 25; j++) {
            printf("%u ", img[i][j]);
        }
        printf("\n");
    }

    return 0;
}


Writing loadImg.c


In [10]:
!gcc loadImg.c

In [11]:
!./a.out

Image Width : 512
Image Height : 512
Image Matrix:
33 68 79 67 84 89 80 69 32 104 116 109 108 62 10 60 104 116 109 108 32 108 97 110 103 
105 116 104 117 98 97 115 115 101 116 115 46 99 111 109 34 32 99 114 111 115 115 111 114 105 
100 105 109 109 101 100 45 57 98 57 97 56 99 57 49 97 99 99 53 46 99 115 115 34 32 
98 46 103 105 116 104 117 98 97 115 115 101 116 115 46 99 111 109 47 97 115 115 101 116 115 
108 61 34 115 116 121 108 101 115 104 101 101 116 34 32 100 97 116 97 45 104 114 101 102 61 
10 32 32 60 108 105 110 107 32 99 114 111 115 115 111 114 105 103 105 110 61 34 97 110 111 
117 111 116 59 99 111 110 100 105 116 105 111 110 115 38 113 117 111 116 59 58 32 38 113 117 
113 117 111 116 59 105 100 38 113 117 111 116 59 58 32 38 113 117 111 116 59 49 54 56 50 
100 38 113 117 111 116 59 58 32 38 113 117 111 116 59 50 48 55 56 53 52 55 48 49 57 
58 32 38 113 117 111 116 59 115 117 98 109 105 116 46 111 114 103 97 110 105 122 97 116 105 
113 117 111 116 59 105 100 38 113 117 111 11

# Richardson-Lucy Algorithm

In [ ]:
%%writefile trial1.cu
#include <stdio.h>
#include <math.h>
#include <stdlib.h>
#define H 6
#define PIX 248
#define NUM 248 * 248
#define SIGMA 25
#ifndef max
#define max( a, b ) ( ((a) > (b)) ? (a) : (b) )
#endif
#ifndef min
#define min( a, b ) ( ((a) < (b)) ? (a) : (b) )
#endif
void dataread(float *y);
int distance(int i, int j, int k, int l);
void lambda(float *y, float *lold, float *lnew, float *p);
void renew(float *lold,float *lnew);
void spread(float *p);
void datasave(float *l);
int main(void){
int i;
float *y, *lold, *lnew, *p, ttt;
y = malloc(NUM * sizeof(float));
lold = malloc(NUM * sizeof(float));
lnew = malloc(NUM * sizeof(float));
p = malloc((2 * (H + 1) * (H + 1)) * sizeof(float));
spread(p);
dataread(y);
dataread(lold);
for(i = 0; i < 40; i++){
lambda(y, lold, lnew, p);
renew(lold, lnew);
printf("%d \n",i);
}
datasave(lnew);
free(p);
free(y);
free(lnew);
free(lold);
return 0;
}
/*READING THE DATA*/
void dataread(float *y){
int i;
FILE *fp;
fp = fopen("mylenna","r");
for(i = 0; i < NUM; i++){
fscanf(fp,"%f ",&y[i]);
}
return;
}
/*DEFINING A NORM*/
int distance(int i, int j, int k, int l){
int d;
d = (i - k) * (i - k) + (j - l) * (j - l);
return d;
}
/*LAMBDA VECTOR*/
void lambda(float *y, float *lold, float *lnew, float *p){
int i1, i2, j1, j2, k1, k2;
float temp, tmp;
for(i1 = 0; i1 < PIX; i1++){
for(i2 = 0; i2 < PIX; i2++){
tmp = 0;
for(j1 = max(0, i1 - H + 1); j1 < min(i1 + H - 1, PIX); j1++){
for(j2 = max(0, i2 - H + 1); j2 < min(i2 + H - 1, PIX); j2++){
temp = 0;
for(k1 = max(0, i1 - H + 1); k1 < min(i1 + H - 1, PIX); k1++){
for(k2 = max(0, i2 - H + 1); k2 < min(i2 + H - 1, PIX); k2++){
temp = temp + lold[k1 * PIX + k2] * p[distance(j1,j2,k1,k2)];
}
tmp = tmp + y[j1 * PIX + j2] * (p[distance(j1,j2,i1,i2)])/temp;
}
}
lnew[i1 * PIX + i2] = lold[i1 * PIX + i2] * tmp;
}
}
return;
}
/*Point Spread Function*/
void spread(float *p){
int i, j;
float temp = 0;
for(i = -H; i <= H; i++){
for(j = -H; j <= H; j++){
p[i * i + j * j] = exp(-((float)(i * i + j * j))/SIGMA);
temp += p[i * i + j * j];
}
}
for(i = -H; i <= H; i++)
for(j = -H; j <= H; j++)
p[i * i + j * j] = p[i * i + j * j]/temp;
return;
}
/*RENEW THE VALUE OF LAMBDA VECTOR*/
void renew(float *lold,float *lnew){
int i;
for(i = 0; i < NUM; i++)
lold[i] = lnew[i];
return;
}
/*SAVING THE DATA*/
void datasave(float *l){
int i, j;
FILE *fp;
fp = fopen("mylenn","w");
for(i = 0; i < PIX; i++){
for(j = 0; j < PIX; j++){
fprintf(fp,"%f ",l[i * PIX + j]);
}
fprintf(fp,"\n");
}
return;
}

Overwriting l9q1a.cu


In [ ]:
!nvcc -arch=sm_37 -gencode=arch=compute_37,code=sm_37 l9q1a.cu -o l9q1a

nvcc warning : The 'compute_35', 'compute_37', 'sm_35', and 'sm_37' architectures are deprecated, and may be removed in a future release (Use -Wno-deprecated-gpu-targets to suppress warning).


In [ ]:
!./l9q1a

Enter the value of m: 2
Enter the value of n: 2
Enter input matrix A:
2 4 5 6
Enter input matrix B:
7 8 3 2
cuda successfully ran!
cuda successfully ran!
Result matrix is:
9	12	
8	8	
